## 🏟️ Credits

- Stride is something Raja first shared: https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/discussion/473011
- The biggest booster in performance is changing triplets to pairs in token reassembling: https://www.kaggle.com/code/nbroad/transformer-ner-baseline-lb-0-854/comments#2659393

Updates and improvements made by Thomas Gamet
This notebook is under the MIT License.
Will test out of striding on inference alone helps my deberta-v3-large trained model
This came from: https://www.kaggle.com/code/valentinwerner/945-deberta-3-base-striding-inference

This particular notebook was forked from a prior version of mine when I realized that it was using more than 1/3rd the memory available to complete a leader board submission. This notebook was rewritten to write its submission file by immediately reassembling inferences and writing them to the submission csv file. The first memory efficiency rewrite seems to have heavy dictionary and Dataset initialization, but I'm not sure (yet) if it makes a significant performance difference (just a note if time limits become an issue).

## Restructured to write predictions immediately to the submission file

With the private dataset expected to be 3 times larger than the public dataset I suspect I need to get the code to run in a third of the free space memory structure. It now seems I need to avoid building up all predictions and the ds_dict as 25% of available RAM is insufficient to maintain all of these structures. It's going to be a bit "ugly" but not really complex.

With the threshold at 0.99.
The deberta-v3-large using 12 of 24 frozen layers, lr=1e-5, 2 data sets, and 2 epochs it scored an LB=0.956
The deberta-v3-large using 6 of 24 frozen layers, lr=5e-6, 2 data sets, and 4 epochs it scored an LB=0.952

Multiple tests were run with ep2lr1e-5ds2fr12seed421+.01testsplit
v4 threshold 0.99 LB=0.941
v5 threshold 0.97 LB=0.947
v6 threshold 0.90 LB=0.947
v7 threshold 0.98 LB=0.946
v8 threshold 0.94 LB=0.948
It seems definitive two epochs with 99% of the training data is not producing good results. Best bet is to try several thresholds with 1 and 3 epochs of training aiming for the LB scores to be like a cross validation score. If that does not work well enough then the full training may require smaller learning rates to do well, or more frozen layers, which is to be determined.

Loading Dataset v6 We now have 3 epochs with ep2lr1e-5ds2fr12seed421+.01td
/kaggle/input/piidd-extending-large-model-starter-model/output/
v9 threshold 0.99 LB=0.920 (checkpoint-4535)
v10 threshold 0.99 LB=0.958 (checkpoint-13605)
v11 threshold 0.97 LB=0.949 (checkpoint-13605)
v12 threshold dynamic LB=0.953 (checkpoint-13605) - highest PII confidence -0.0075
v13 threshold dynamic LB=0.952 (checkpoint-13605) - highest PII confidence -0.0100
New strategy => pick the model epoch that gives the best LB for a threshold of .99

Loaded Dataet v7 /kaggle/input/piidd-extending-large-model-starter-
ep4lr1e-5ds3fr12seed421+.01td epochs 4
model/output/checkpoint-26920
v14 threshold 0.99 LB=0.932
model/output/checkpoint-20190
v15 error running without GPU, need to wait for it to faill
v16 threshold 0.99 LB=0.940
model/output/checkpoint-13460
v17 threshold 0.99 LB=0.951
model/output/checkpoint-6730
v18 threshold 0.99 LB=0.930

ep#lr2e-6ds3fr12seed42+.1td-of5ep
Loaded Dataet v8 /kaggle/input/piidd-extending-large-model-starter-
model/output/checkpoint-6118
v19 threshold 0.99 LB=0.924
model/output/checkpoint-12236
v20 threshold 0.99 LB=0.945
model/output/checkpoint-18354
v21 threshold 0.99 LB=0.933
model/output/checkpoint-24472
v22 threshold 0.99 LB=0.935
model/output/checkpoint-30590
v23 was not a submission
v24 threshold 0.99 LB=0.934

ep#lr2e-5ds3fr12seed42+.1td+of3ep
Loaded Dataset v9 /kaggle/input/piidd-extending-large-model-starter-
model/output/checkpoint-6118
v25 threshold 0.99 LB=0.942
model/output/checkpoint-12236
v26 threshold 0.99 LB=0.948
model/output/checkpoint-18354
v27 threshold 0.99 LB=0.945

Loaded Dataset v10 
ep#lr5e-6ds4fr12seed42td.1tb4vb8
checkpoint-10527 which is ep3
v28 threshold 0.99 LB=0.939
checkpoint-14036 which is ep4
v29 threshold 0.99 LB=0.942

Trained v11 v11 ep4lr9e-6ds4fr18seed42td.1tb4vb8
checkpoint-14036 which is ep4 with the highest training F1 score
v30 threshold 0.99 LB=0.938

Also, an experiment or two with 2048 and 720 for INFERENCE_MAX_LENGTH is needed.
v34 using training v11 ep3lr1e-5ds2nfr12seed421td.1 threshold 0.99
LB at maxlen=2048 was 0.953
LB at maxlen=720 was 0.951
LB at maxlen=1024 was 0.952

v35 using training v12 ep5lr1e-5ds2nfr12seed421td.01 threashold 0.99
LB at maxlen=1024 with epoch 4 of 5 was 0.947
v36 LB at maxlen=1024 with epoch 3 of 5 was 0.948
v37 LB at maxlen=1024 with epoch 5 of 5 was 0.949
v38 LB at maxlen=1024 with epoch 2 of 5 was 0.958
v39 LB at maxlen=1024 with epoch 1 of 5 was 0.963 (short lived winner)
v40 LB at maxlen=2048 with epoch 1 of 5 was 0.965 (new winner)

The aim here is a similar LB with 3 different training datasets to see if they can form an ensemble. Theory, training with all datasets together decreases generalization. They high learning rate, compared to what the model specs say it was trained at, are likely the reason why it both 'learns' in an epoch or two, and why it soon starts loosing performance (overfits on its training data and loses generalization).
Training v22 ep2lr1e-5ds3nfr12seed421td.01
Concern, I have to basically negate the threshold to 0.001 to get it to filter down answers.
v41 LB at maxlen=2048 with epoch 1 of 2 was 0.906
v42 LB at maxlen=2048 with epoch 2 of 2 was 0.915
This dataset is not promissing.

Trying with v14 in the model dataset v23ep3lr1e-5ds2nfr12seed421td.01
v43 LB at maxlen-2048 with epoch 1 of 3 was 0.934
v44 LB at maxlen-2048 with epoch 2 of 3 was 0.948
v45 LB at maxlen-2048 with epoch 3 of 3 was 0.954
We need more epochs to see if at which one it reaches the point of an overfit.
v46 LB at maxlen-2048 with epoch 4 of 6 was 0.941 which shows we have begun overfitting

Next move, rebuild v12 with slight alterations to see if it helps in an ensemble.

v46 ran with an error - wrong model checkpoint selection
v47 confirming we rerun v40 with Trained model v12 /kaggle/input/piidd-extending-large-model-starter-model
v48 seeing if v1 of trn2-model is like v12 above (two more epochs to try) -> LB=0.94
That is different, but there are two more epochs to test as if an LB was a CV.
v49 with v1 epoch 2 -> LB=0.958
v50 with v1 epoch 3 -> LB=? (mislabeled run, it really is epoch 3's model output). 

**** New Direct CSV generation on a per document's prediction.

v1 LB=0.963 recreating Training v12 ep5lr1e-5ds2nfr12seed421td.01 threashold 0.99 results but with 16GB reserved for the private dataset size expansion on a max length 1024 tokens
v2 LB-0.965 provided all of V1 works but with 2048 allowed. It is also good to note that I believe the memory footprint is now much lower so if this works OOM should no longer be a major concern.
v3 builds train v12 ep5lr1e-5ds2nfr12seed421td.01, threashold 0.99, maxlen=2048 for the competition at an LB=0.965 with full RAM available.
v4 builds on trn2 v02-ep1of4lr1e-5ds2nfr12seed9973tr.01, threshold 0.99, maxlen-2048 for competition at an LB=0.952 with full RAM and epoch 0.
v5 builds on trn2 trn2v2-ep2of4lr1e-5ds2nfr12seed9973td.01, threshold 0.99, maxlen-2048 for competition at an LB=0.945 with full RAM and epoch 1.
v6 builds on trn2 trn2v2-ep3of4lr1e-5ds2nfr12seed9973td.01, threshold 0.99, maxlen-2048 for competition at an LB=0.959 with full RAM and epoch 2.
v7 builds on trn2 trn2v2-ep4of4lr1e-5ds2nfr12seed9973td.01, threshold 0.99, maxlen-2048 for competition at an LB=0.947 with full RAM and epoch 2.

v8 Ensemble test yielding LB=0.967 new high
  a. builds train v12 ep5lr1e-5ds2nfr12seed421td.01, threashold 0.99, maxlen=2048
  b. trn2 trn2v2-ep3of4lr1e-5ds2nfr12seed9973td.01, threshold 0.99, maxlen-2048
Also, this run will remove 20GB of RAM to confirm more than enough memory exists.
v9 is the same ensemble with all memory available.

v10 Ensemble test yielding LB=0.968 new high
  a. builds train v12 ep5lr1e-5ds2nfr12seed421td.01, threashold 0.99, maxlen=2048
  b. trn2 trn2v1-ep3of3lr1e-5ds2nfr12seed4221td.01, threshold 0.99, maxlen-2048
  
Single model run with trn2v3-ep4lr1e-5ds2nfr12seed8128td.01
v11 with Epoch 1 produces LB=0.910
v12 with Epoch 2 produces LB=0.962
v13 with Epoch 3 produces LB=0.952

v14 Ensemble test yielding LB=0.967
  a. builds train v12 ep5lr1e-5ds2nfr12seed421td.01, threashold 0.99, maxlen=2048
  b. trn2 trn2v3-ep2of3lr1e-5ds2nfr12seed8128td.01, threshold 0.99, maxlen=2048
  
v15 Ensemble test for memory footprint yielding LB=0.965
  a. builds train v12 ep5lr1e-5ds2nfr12seed421td.01, threashold 0.99, maxlen=2048
  b. trn2 trn2v2-ep3of3lr1e-5ds2nfr12seed9973td.01, threshold 0.99, maxlen-2048
  c. trn2 trn2v2-ep1of3lr1e-5ds2nfr12seed9973td.01, threshold 0.99, maxlen-2048
It finished in 2.75 hours with 18GB of memory reserved for the private set size.
The LB=0.965 (which is fine as the individual models had 0.965, 0.959, 0.952).
Though this will be close with the private data size being 3x the public data size I will try at least one three model run maximizing the LB. The other two selections will use good, but not optimal, LBs based upon multiple prior experiences where the mid to upper mid CV's tend to produce the best LB scores, and I'm pretty much treating the LB like a CV score.

The [Best-Trn2] contains:
v1 is ep3lr1e-5ds2nfr12seed4221td.01 [https://www.kaggle.com/code/thomasgamet/piidd-extending-large-model-starter-trn2 v1] epoch 3 LB=0.961
v2 is ep4lr1e-5ds2nfr12seed9973td.01 [https://www.kaggle.com/code/thomasgamet/piidd-extending-large-model-starter-trn2 v2] epoch 3 LB=0.959
v3 is ep4lr1e-5ds2nfr12seed8128td.01 [https://www.kaggle.com/code/thomasgamet/piidd-extending-large-model-starter-trn2 v3] epoch 2 LB=0.962 which is Trn2 v3

v16 was an invalid build attempt with the original model not correctly pinned.

v17 Ensemble test with trshold 0.99, maxlen=2048, -> yielding LB=0.965
  a. builds train v12 ep5lr1e-5ds2nfr12seed421td.01, threashold 0.99, maxlen=2048
  b. Model-trn2-v1
  c. Model-trn2-v2
  
v18 Ensemble test with trshold 0.99, maxlen=2048, -> yielding LB=0.965
  a. builds train v12 ep5lr1e-5ds2nfr12seed421td.01, threashold 0.99, maxlen=2048
  b. Model-trn2-v1
  c. Model-trn2-v3
  
v19 Ensemble test with trshold 0.99, maxlen=2048, -> yielding LB=0.965
  a. builds train v12 ep5lr1e-5ds2nfr12seed421td.01, threashold 0.99, maxlen=2048
  b. Model-trn2-v2
  c. Model-trn2-v3
  
v20 Ensemble test with trshold 0.99, maxlen=2048, -> yielding LB=0.961
  a. Model-trn2-v1
  b. Model-trn2-v2
  c. Model-trn2-v3

Well, this was disappointing, a three model ensemble failed to produce an improved score, but with that said, nont all models have equal weight. A few more tries with weights on v18 which combines the top three individual models.

v21 Ensemble test with trshold 0.99, maxlen=2048, -> yielding LB=0.965
  a. 0.4 builds train v12 ep5lr1e-5ds2nfr12seed421td.01, threashold 0.99, maxlen=2048
  b. 0.3 Model-trn2-v1
  c. 0.3 Model-trn2-v3
  
v22 Ensemble test with trshold 0.99, maxlen=2048, -> yielding LB=0.969
  a. 0.5 builds train v12 ep5lr1e-5ds2nfr12seed421td.01, threashold 0.99, maxlen=2048
  b. 0.25 Model-trn2-v1
  c. 0.25 Model-trn2-v3
  
v23 Ensemble test with trshold 0.99, maxlen=2048, -> yielding LB=0.969
  a. 0.5 builds train v12 ep5lr1e-5ds2nfr12seed421td.01, threashold 0.99, maxlen=2048
  b. 0.2 Model-trn2-v1
  c. 0.3 Model-trn2-v3
  
v24 was v23 with maxlen on inference set to 4096 and testing for OOM. It ran to completion, and on time, but scored only 0.969.

v25 Ensemble test with trshold 0.99, maxlen=4096, -> yielding LB=0.969
  a. 0.5 builds train v12 ep5lr1e-5ds2nfr12seed421td.01, threashold 0.99, maxlen=4096
  b. 0.25 Model-trn2-v1
  c. 0.25 Model-trn2-v3
  
Left weights at .5 .25 .25
v26 Ensemble test with trshold 0.970, maxlen=2048, -> yielding LB=0.967
v27 Ensemble test with trshold 0.995, maxlen=2048, -> yielding LB=0.967

v28 Ensemble test with trshold 0.970, maxlen=2048, w.34.33.33 -> yielding LB=0.967
v29 Ensemble test with trshold 0.995, maxlen=2048, w.34.33.33 -> yielding LB=0.967

OK, it seems we are stuck and for the LB a threshold of 0.99 has proven the most effective.

Tweeked Trn2 (https://www.kaggle.com/code/thomasgamet/piidd-extending-large-model-starter-trn2 v4) until it could fine tune with TRAINING_MAX_LENGTH = 1024 (previously OOM occurred at values over 720). Let's see this matchup to the hidden layer size improves the models produced  to yield better inferencing results.

ep4lr1e-5ds2nfr12seed8128td.01GPUP100tml1k as an "ensemble" of one.... my mistake
v30 Ensemble test with checkpoint-4702 thrshold 0.99, maxlen=2048, -> yielding LB=0.923
v31 Ensemble test with checkpoint-9404 thrshold 0.99, maxlen=2048, -> yielding LB=0.956
v32 Ensemble test with checkpoint-14106 thrshold 0.99, maxlen=2048, -> yielding LB=0.955
v33 Ensemble test with checkpoint-18808 thrshold 0.99, maxlen=2048, -> yielding LB=0.954

v34 Ensemble test with checkpoint-9404 threshold 0.90, maxlen=2048 -> yielding LB=0.948
Reducing the threshold increases the nummber of tokens seen as class "O" (not PII) which means positive PII detection decreases, and this lowers the score even though recall is weights over precision. 

Tweeked Trn2 (https://www.kaggle.com/code/thomasgamet/piidd-extending-large-model-starter-trn2 v5) until it could fine tune with TRAINING_MAX_LENGTH = 1024 (previously OOM occurred at values over 720). Let's see this matchup to the hidden layer size improves the models produced  to yield better inferencing results.

ep4lr1e-5ds2nfr12seed9973td.01P100len1k as an "ensemble" of one.... my mistake
v35 - error, no GPU, also v36 and v37 will likely be redone with threshold 0.99
v36 Ensemble test with checkpoint-4702 threshold 0.9, maxlen=2048, -> yielding LB=0.944
v37 Ensemble test with checkpoint-9404 threshold 0.9, maxlen=2048, -> yielding LB=0.949
v38 Ensemble test with checkpoint-4702 threshold 0.99, maxlen=2048, -> yielding LB=0.917
v39 Ensemble test with checkpoint-9404 threshold 0.90, maxlen=2048, -> yielding LB=0.955
v40 Ensemble test with checkpoint-14106 threshold 0.99, maxlen=2048, -> yielding LB=0.955
v41 Ensemble test with checkpoint-18808 threshold 0.99, maxlen=2048, -> yielding LB=0.951

This is super disturbing. Lower LB scores despite using longer strings of training tokens. Tweeked Trn2 (https://www.kaggle.com/code/thomasgamet/piidd-extending-large-model-starter-trn2 v6) until it could fine tune with TRAINING_MAX_LENGTH = 512. Yes,this is an attempt to see if it helps to go to a smaller size. Assuming this does not help, I'll try to redesign training to chuckify the essays.

v42 Ensemble test with checkpoint-4702 threshold 0.99, maxlen=2048 --> yielding LB=0.910
v43 Ensemble test with checkpoint-9404 threshold 0.99, maxlen=2048 --> yielding LB=0.934
Likely a no go situation, with just the first 512 tokens of esseys usedthe start of interencing is scoring low.

Tweeked Trn2 (https://www.kaggle.com/code/thomasgamet/piidd-extending-large-model-starter-trn2 v7) until it could fine tune with TRAINING_MAX_LENGTH = 512 chunks with all tokens included in training (but not smart chunking - as the last chunk gets a reminder without consideration of its size).

v44 Ensemble test with ep1&9230&chn512 threshold 0.99, maxlen=2048 --> yielding LB=0.937
v45 Ensemble test with ep2&18460&chn512 threshold 0.99, maxlen=2048 --> yielding LB=0.954
v46 Ensemble test with ep3&27690&chn512 threshold 0.99, maxlen=2048 --> yielding LB=0.937

Tweeked Trn2 (https://www.kaggle.com/code/thomasgamet/piidd-extending-large-model-starter-trn2 v8) until it could fine tune with TRAINING_MAX_LENGTH = 720 chunks and lr=5e-6 with all tokens included in training (but not smart chunking - as the last chunk gets a reminder without consideration of its size).

v47 Ensemble test with ep2&14504&chn512 threshold 0.99, maxlen=2048 --> yielding LB=0.955
v48 Ensemble test with ep3&21756&chn512 threshold 0.99, maxlen=2048 --> yielding LB=0.954
v49 Ensemble test with ep4&21756&chn512 threshold 0.99, maxlen=2048 --> yielding LB=0.955

Tweeked Trn2 (https://www.kaggle.com/code/thomasgamet/piidd-extending-large-model-starter-trn2 v9) until it could fine tune with TRAINING_MAX_LENGTH = 720 chunks and lr=1e-5 with all tokens included in training. Also going to try to double the batch size and allow 10% for validation to be more 'useful'. Best score of the past single model v11 of original training used a .1 split and showed worse training losses with a larger batch size. This is trn2 notebook version 12. 51Single-private-readyTrn2m12-c720P100tr.99ep1
Epoch	Training Loss	Validation Loss	Recall	Precision	F1
1	0.002100	0.001067	0.979037	0.965544	0.978511
2	0.000600	0.000585	0.989130	0.975996	0.988619
3	0.000300	0.000505	0.991718	0.987374	0.991551
4	0.000100	0.000519	0.990942	0.990173	0.990912
v50 model-v9 test with ep1 threshold 0.99, maxlen=2048 --> yielding LB=0.933
v51 model-v9 test with ep2 threshold 0.99, maxlen=2048 --> yielding LB=0.961 *
v52 model-v9 test with ep3 threshold 0.99, maxlen=2048 --> yielding LB=0.960
v53 model-v9 test with ep4 threshold 0.99, maxlen=2048 --> yielding LB=0.944

Epoch	Training Loss	Validation Loss	Recall	Precision	F1
1	0.002100	0.001264	0.974164	0.969568	0.973987
2	0.000800	0.000901	0.978905	0.975437	0.978771
3	0.000300	0.000816	0.981749	0.986895	0.981946
4	0.000200	0.000898	0.982934	0.990683	0.983230
Trn2 version 13 ep4lr1e-5ds2nfr12seed421td.1P100chn720b4
v54 model-v10 test with ep1 threshold 0.99, maxlen=2048 --> yielding LB=0.950
v55 model-v10 test with ep2 threshold 0.99, maxlen=2048 --> yielding LB=0.953
v56 model-v10 test with ep3 threshold 0.99, maxlen=2048 --> yielding LB=0.958 *
v57 model-v10 test with ep4 threshold 0.99, maxlen=2048 --> yielding LB=0.956

Trn2 version 14 ep4lr1e-5ds2nfr12seed8128td.1P100chn720b4
V??Single-private-readyTrn2m14-c720P100tr.99ep?
Epoch	Training Loss	Validation Loss	Recall	Precision	F1
1	0.001600	0.000959	0.978705	0.965474	0.978190
2	0.000400	0.001077	0.978916	0.982646	0.979059
3	0.000300	0.000882	0.982079	0.991908	0.982453
4	0.000100	0.000940	0.983344	0.987090	0.983487
v60 model-v11 test with ep1 threshold 0.99, maxlen=2048 --> yielding LB=0.953
v61 model-v11 test with ep2 threshold 0.99, maxlen=2048 --> yielding LB=0.963 *
v62 model-v11 test with ep3 threshold 0.99, maxlen=2048 --> yielding LB=0.946
v63 model-v11 test with ep4 threshold 0.99, maxlen=2048 --> yielding LB=0.958

All Pairs
v58 Ensemble test tv12&trn2-v9 with trshold 0.99, maxlen=2048, -> yielding LB=0.969
  a. 0.5 builds train v12 ep5lr1e-5ds2nfr12seed421td.01
  b. 0.5 Model-trn2-v9
v59 Ensemble test tv12&trn2-v10 with trshold 0.99, maxlen=2048, -> yielding LB=0.962
  a. 0.5 builds train v12 ep5lr1e-5ds2nfr12seed421td.01
  b. 0.5 Model-trn2-v10
v66 Ensemble test tv12&trn2-v11 with trshold 0.99, maxlen=2048, -> yielding LB=0.968
  a. 0.5 builds train v12 ep5lr1e-5ds2nfr12seed421td.01
  b. 0.5 Model-trn2-v11

All Triplets
Top three modes by build setup: v64 Ensemble test tv12&best-trn2-v3&trn2-v11 LB=0.968
A. 0.50 on V12 of /kaggle/input/piidd-extending-large-model-starter-model/output/checkpoint-4702 ep1
B. 0.20 on Best-Models-Trn2-v3 at 0.962 => /kaggle/input/piidd-extending-large-model-starter-best-trn2/output/model-trn2-v3-checkpoint-9404 ep1 Trn2 v12
C. 0.30 V11 of /kaggle/input/piidd-extending-large-model-starter-model-trn2/output/checkpoint-6594 ep2 of Trn2 build v14

Top three modes by build setup: v65 Ensemble test tv12&best-trn2-v3&trn2-v11 LB=0.968
A. 0.34 on V12 of /kaggle/input/piidd-extending-large-model-starter-model/output/checkpoint-4702 ep1
B. 0.33 on Best-Models-Trn2-v3 at 0.962 => /kaggle/input/piidd-extending-large-model-starter-best-trn2/output/model-trn2-v3-checkpoint-9404 ep1 Trn2 v12
C. 0.33 V11 of /kaggle/input/piidd-extending-large-model-starter-model-trn2/output/checkpoint-6594 ep2 of Trn2 build v14

Top three modes by build setup: v65 Ensemble test tv12&best-trn2-v3&trn2-v11 LB=0.968
A. 0.4 on V12 of /kaggle/input/piidd-extending-large-model-starter-model/output/checkpoint-4702 ep1
B. 0.2 on Best-Models-Trn2-v3 at 0.962 => /kaggle/input/piidd-extending-large-model-starter-best-trn2/output/model-trn2-v3-checkpoint-9404 ep1 Trn2 v12
C. 0.4 V11 of /kaggle/input/piidd-extending-large-model-starter-model-trn2/output/checkpoint-6594 ep2 of Trn2 build v14

Reality Check, the private leader board has 75% of the data, which means it will take 3 times longer to complete inferencing than the public leader board data set. Bottom line, only the ensembles of 2 models will (or are expected) to complete on time.

Let's run all pairs for the Best2 of Trn2 which are the best individual scoring models of Trn2 which used 720 training length, all data in chunks: ep5lr1e-5ds2nfr12seed####td.1

Pair v68 best2-trn2-v9,best2-trn2-v10 has LB=0.964
Pair v69 best2-trn2-v9,best2-trn2-v11 has LB=0.961
Pair v70 best2-trn2-v10,best2-trn2-v11 has LB=0.967

Test to see how well a single xsmall deberta model tuned with our data can perform.
v71 uses model-trn2-v12 (trn2-v21) threshold=0.51 epoch 4 to get LB=0.914
sample_submission had 25 true positives 0 false negatives and 2 false positives
v72 uses model-trn2-v12 (trn2-v21) threshold=0.96 epoch 4 to get LB=0.942
sample_submission had 25 true positives 0 false negatives and 8 false positives
v73 uses model-trn2-v12 (trn2-v21) threshold=0.86 epoch 4 to get LB=0.
sample_submission had 25 true positives 0 false negatives and 5 false positives
v74 uses model-trn2-v12 (trn2-v21) threshold=0. epoch 4 to get LB=0.
sample_submission had 25 true positives 0 false negatives and 11 false positives


My Training Notebook is at: https://www.kaggle.com/code/thomasgamet/piidd-extending-large-model-starter-training

This inferencing notebook's key ideas were originally built to work with...

## 🛑 Wait a second - maybe also look at the training
- My training notebook (containing equally many emojis) is here: I would love an upvote if you use the notebook or learned something new!
- https://www.kaggle.com/code/valentinwerner/915-deberta3base-training

# Reserve 25% for models and system, then 25% for the public dataset.
The other 50% is taken away for a test to minimize the risk of an OOM exception.
Basically, with 75% of the data in the private set, it will be 3x the size of the public
data set. A single model with the training set, with 16GB pulled out, is at 71% memory used which leaves just about the right amount that must suffice for the public test set.

Reserving half the memory is probably overkill, given that the redesign to write the CSV file in batches of 50 documents and collect garbadge between the batches likely means the only linear growth will be in the initial Dataset object to feed the inferencing process (which remains done in one large batch of all json objects in the test set.) The rewrite seems to have little impact on wall time and so I will halt optimzation of the loops in favor of exploring inteferencing and ensemble based improvements.

****# Comment out and make a Markup for contest submission
#useToTestForPrivateSetSize = bytearray(int(20 * (1024 ** 3))) synatx error in case it is both uncommented and changed to a code block by accident

## Initial parameters and models

In [1]:
INFERENCE_MAX_LENGTH=2048 # 1024
STRIDE=384

# Note that training a model with stride, such as: https://www.kaggle.com/code/thedrcat/pii-data-detection-train-with-w-b
# will also improve performance
# the tokenizer will be taken from only the first model in a set of models
# best choice is a model that differs only on the splits used.
## model_path = "/kaggle/input/piidd-extending-large-model-starter-model-trn2/output/checkpoint-4702"

# Use for ensembles, and be sure to pin the needed version before running the code
##model_path = "/kaggle/input/piidd-extending-large-model-starter-model/output/checkpoint-4702" # pin v12 LB=0.965 at tr.99 maxlen2048
#model_path = "/kaggle/input/piidd-extending-large-model-starter-trn2/output/checkpoint-367" # xsmall with issues built with 10% of the data
model_path = "/kaggle/input/piidd-extending-large-model-starter-model-trn2/output/checkpoint-13920"

# using the model path below to test a new trn2 bbuild with 1024 byte hidden/inference length (was 720)
#model_path = "/kaggle/input/piidd-extending-large-model-starter-model-trn2/output/checkpoint-13188"

model_path2 = "/kaggle/input/piidd-extending-large-model-starter-best2-trn2/output/model-trn2-v11-checkpoint-6594"
#model_path2 = "/kaggle/input/piidd-extending-large-model-starter-best-trn2/output/model-trn2-v2-checkpoint-14106"
model_path3 = "/kaggle/input/piidd-extending-large-model-starter-best2-trn2/output/model-trn2-v11-checkpoint-6594" # avoiding 3 models due to time constraints when we must inference with 3x the public leaderboards data in 9 hours or less

# Comment out when we have modified this for a single model run
#model_path = "/kaggle/input/piidd-extending-large-model-starter-model-trn2/output/checkpoint-14106"

In [2]:
import json
import argparse
from itertools import chain
import pandas as pd
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import Dataset 
import numpy as np
import gc

# Might revisit, but for now HF datasets module is not supported by the TPU / CUDA
# I would need to rewrite without datasets to import Dataset
# Comment out the import and device setting when using the GPU P100 or T4x2
#import torch # we will use this to get some more GPU time from the TPUs
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# also, look for .to(device) and comment out those lines when not using the TPUs

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## ♟️ Data Loading & Data Tokenization
- This tokenizer is actually special, comparing to usual NLP challenges
- inference tokenizer is a bit different than training tokenizer, because we don't have labels

In [3]:
def tokenize(example, tokenizer):
    text = []
    token_map = []
    
    idx = 0
    
    for t, ws in zip(example["tokens"], example["trailing_whitespace"]):
        text.append(t)
        token_map.extend([idx]*len(t))
        if ws:
            text.append(" ")
            token_map.append(-1)
            
        idx += 1
        
    tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=INFERENCE_MAX_LENGTH, stride=STRIDE, return_overflowing_tokens=True)
        
    return {
        **tokenized,
        "token_map": token_map,
    }

In [4]:
data = json.load(open("/kaggle/input/pii-detection-removal-from-educational-data/test.json"))

ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [x["document"] for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
})

tokenizer = AutoTokenizer.from_pretrained(model_path)
ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer}, num_proc=2)

#0:   0%|          | 0/5 [00:00<?, ?ex/s]

#1:   0%|          | 0/5 [00:00<?, ?ex/s]

## 🏋🏻‍♀️ Trainer Class based on the trained model

In [5]:
model = AutoModelForTokenClassification.from_pretrained(model_path)

# model.to(device) # used only with CUDA to access the TPU

collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
args = TrainingArguments(
    ".", 
    per_device_eval_batch_size=1, 
    report_to="none",
)
trainer = Trainer(
    model=model, 
    args=args, 
    data_collator=collator, 
    tokenizer=tokenizer,
)

# This needs a second model build off the exact same LLM base (large in this case).
It must be just another run as it must have the same tokenizer 

In [6]:
model2 = AutoModelForTokenClassification.from_pretrained(model_path2)

# model.to(device) # used only with CUDA to access the TPU

collator2 = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
args = TrainingArguments(
    ".", 
    per_device_eval_batch_size=1, 
    report_to="none",
)
trainer2 = Trainer(
    model=model2, 
    args=args, 
    data_collator=collator2, 
    tokenizer=tokenizer,
)

In [7]:
model3 = AutoModelForTokenClassification.from_pretrained(model_path3)

# model.to(device) # used only with CUDA to access the TPU

collator3 = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
args = TrainingArguments(
    ".", 
    per_device_eval_batch_size=1, 
    report_to="none",
)
trainer3 = Trainer(
    model=model3, 
    args=args, 
    data_collator=collator3, 
    tokenizer=tokenizer,
)

### Striding functions

As using the stride give an overlap in tokens, these have to be removed (either pick one side of the stride or average them, ...).

In [8]:
def backwards_map_preds(sub_predictions, max_len):
    if max_len != 1: # nothing to map backwards if sequence is too short to be split in the first place
        if i == 0:
            # First sequence needs no SEP token (used to end a sequence)
            sub_predictions = sub_predictions[:,:-1,:]
        elif i == max_len-1:
            # End sequence needs to CLS token + Stride tokens 
            sub_predictions = sub_predictions[:,1+STRIDE:,:] # CLS tokens + Stride tokens
        else:
            # Middle sequence needs to CLS token + Stride tokens + SEP token
            sub_predictions = sub_predictions[:,1+STRIDE:-1,:]
    return sub_predictions

def backwards_map_(row_attribute, max_len):
    # Same logics as for backwards_map_preds - except lists instead of 3darray
    if max_len != 1:
        if i == 0:
            row_attribute = row_attribute[:-1]
        elif i == max_len-1:
            row_attribute = row_attribute[1+STRIDE:]
        else:
            row_attribute = row_attribute[1+STRIDE:-1]
    return row_attribute

In [9]:
%%time
config = json.load(open(Path(model_path) / "config.json"))
id2label = config["id2label"]
gcnow = 0
giverowid = 0
ansrow = 0

print("Writing submission.csv with pauses for GC to work.")
with open("submission.csv", 'w') as csv_file:
    submission = pd.DataFrame(columns= ["row_id", "document", "token", "label"])
    submission.to_csv(csv_file, header=True, index=False)

    preds = []
    
    for row in ds:
        if gcnow % 50 == 49:
            _ = gc.collect()
        gcnow += 1
        
        # keys that need to be re-assembled
        row_preds = []
        row_offset = []
        ds_dict = {
            "document":[],
            "token_map":[],
            "offset_mapping":[],
            "tokens":[]
        }
        
        # Build a dictionary for reassembling the predictions and call for the inference
        for i, y in enumerate(row["offset_mapping"]):
            # create new datasset for each of of the splits per document
            x = Dataset.from_dict({
                "token_type_ids":[row["token_type_ids"][i]],
                "input_ids":[row["input_ids"][i]],
                "attention_mask":[row["attention_mask"][i]],
                "offset_mapping":[row["offset_mapping"][i]]
            })
            # predict for that split
            pred = trainer.predict(x).predictions
            #pred = (trainer.predict(x).predictions + trainer2.predict(x).predictions)/2.0
            #pred = 0.4 * trainer.predict(x).predictions + 0.3 * trainer2.predict(x).predictions + 0.4 * trainer3.predict(x).predictions
            
            # removing the stride and additional CLS & SEP that are created
            row_preds.append(backwards_map_preds(pred, len(row["offset_mapping"])))
            row_offset += backwards_map_(y, len(row["offset_mapping"]))
   
        # Complete the prediction
        preds_final = []
        p_concat = np.concatenate(row_preds, axis = 1)
        predictions_softmax = np.exp(p_concat) / np.sum(np.exp(p_concat), axis = 2).reshape(p_concat.shape[0],p_concat.shape[1],1)
        p_concat = p_concat.argmax(-1) # the highest probability class of PII predicted
        predictions_without_O = predictions_softmax[:,:,:12].argmax(-1)
        O_predictions = predictions_softmax[:,:,12] # the probability that it is not PII (aka class "O")
        # this threshold seems more of an art than a science unless I can find a function to estimate or optimize it
        threshold = 0.97 # 0.97 # 0.99 # the threshold of confidence below which we assume the prediction is a class of PII
        preds_final.append(np.where(O_predictions < threshold, predictions_without_O , p_concat))

        ds_dict["document"].append(row["document"])
        ds_dict["tokens"].append(row["tokens"])
        ds_dict["token_map"].append(row["token_map"])
        ds_dict["offset_mapping"].append(row_offset)
        ds = Dataset.from_dict(ds_dict)
        pairs = []
        document, token, label, token_str = [], [], [], []

        for p, token_map, offsets, tokens, doc in zip(preds_final, ds["token_map"], ds["offset_mapping"], ds["tokens"], ds["document"]):
            for token_pred, (start_idx, end_idx) in zip(p[0], offsets):
                label_pred = id2label[str(token_pred)]

                if start_idx + end_idx == 0: continue

                if token_map[start_idx] == -1:
                    start_idx += 1

                # ignore "\n\n"
                while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
                    start_idx += 1

                if start_idx >= len(token_map): break

                token_id = token_map[start_idx]

                # ignore "O" predictions and whitespace preds
                if label_pred != "O" and token_id != -1:
                    pair=(doc, token_id)
                    if pair not in pairs:
                        pairs.append(pair) # we will need to remember the pairing
                        rowstr = str(giverowid)+","+str(doc)+","+str(token_id)+","+label_pred+"\n"
                        print(ansrow, ", ", rowstr)
                        ansrow += 1
                        csv_file.write(rowstr)
                        giverowid += 1
        
_ = gc.collect()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Writing submission.csv with pauses for GC to work.


0 ,  0,7,4,B-NAME_STUDENT

1 ,  1,7,6,B-NAME_STUDENT

2 ,  2,7,9,B-NAME_STUDENT

3 ,  3,7,10,I-NAME_STUDENT

4 ,  4,7,482,B-NAME_STUDENT

5 ,  5,7,483,I-NAME_STUDENT

6 ,  6,7,738,B-NAME_STUDENT

7 ,  7,7,741,B-NAME_STUDENT

8 ,  8,7,742,I-NAME_STUDENT



9 ,  9,10,0,B-NAME_STUDENT

10 ,  10,10,1,I-NAME_STUDENT

11 ,  11,10,464,B-NAME_STUDENT

12 ,  12,10,465,I-NAME_STUDENT



13 ,  13,16,4,B-NAME_STUDENT

14 ,  14,16,5,I-NAME_STUDENT



...

21 ,  21,86,6,B-NAME_STUDENT

22 ,  22,86,7,I-NAME_STUDENT



23 ,  23,93,0,B-NAME_STUDENT

24 ,  24,93,1,I-NAME_STUDENT



25 ,  25,104,7,B-NAME_STUDENT

26 ,  26,104,8,B-NAME_STUDENT

27 ,  27,104,9,I-NAME_STUDENT



28 ,  28,112,5,B-NAME_STUDENT

29 ,  29,112,6,I-NAME_STUDENT



30 ,  30,123,32,B-NAME_STUDENT

31 ,  31,123,33,I-NAME_STUDENT

32 ,  32,123,35,I-NAME_STUDENT

33 ,  33,123,38,B-NAME_STUDENT

34 ,  34,123,1509,B-URL_PERSONAL

35 ,  35,123,1512,B-URL_PERSONAL

36 ,  36,123,1514,B-URL_PERSONAL

CPU times: user 1.33 s, sys: 117 ms, total: 1.44 s
Wall time: 1.81 s


In [10]:
import psutil

# Get the memory details
memory = psutil.virtual_memory()

total_memory = memory.total / (1024 ** 3)  # Convert bytes to GB
print(f"Total RAM: {total_memory:.2f} GB")

available_memory = memory.available / (1024 ** 3)  # Convert bytes to GB
print(f"Available RAM: {available_memory:.2f} GB")

used_memory = memory.used / (1024 ** 3)  # Convert bytes to GB
print(f"Used RAM: {used_memory:.2f} GB")

memory_usage_percent = memory.percent
print(f"Memory Usage: {memory_usage_percent}%")


Total RAM: 31.36 GB
Available RAM: 27.61 GB
Used RAM: 3.28 GB
Memory Usage: 11.9%
